Parte introduttiva:
 - configurazione
 

In [24]:
# Installazione delle librerie tramite conda
""" conda install -c conda-forge pytesseract pdf2image opencv pillow spacy pandas """
# Installazione delle librerie tramite pip
""" pip install pdfplumber transformers torch tensorflow """
# Scarica il modello di SpaCy:
""" python -m spacy download en_core_web_sm """

# TUTORIAL INSTALLAZIONE E CONFIGURAZIONE tessseract odcr: https://www.youtube.com/watch?v=GMMZAddRxs8&ab_channel=DataSlinger
# Cos'è Tesseract OCR??
# Tesseract è un motore OCR open source sviluppato da Google, in grado di riconoscere il testo in immagini e PDF.

# Per camelot:
# conda install -c conda-forge camelot-py

# Cos'è SpaCy?? 
# https://spacy.io/models , libreira funzioni NLP (I MODULI DI SPACY vanno instalalti, esempio python -m spacy download it_core_news_lg):

# Caratteristiche Principali
# Tokenizzazione: SpaCy può suddividere un testo in parole, punteggiatura e altri elementi (token).
# Part-of-Speech Tagging (POS): Assegna una categoria grammaticale (nome, verbo, aggettivo, ecc.) a ciascun token.
# Dependency Parsing: Analizza le relazioni grammaticali tra parole in una frase.
# Named Entity Recognition (NER): Riconosce e classifica entità nominate (come persone, organizzazioni, luoghi, date) nel testo.
# Lemmatizzazione: Riduce le parole alle loro forme base o radici (lemma).
# Supporto per più lingue: SpaCy supporta molte lingue diverse e include modelli pre-addestrati per diverse lingue.
# Vettori di parole: Supporta modelli di word embedding come Word2Vec, GloVe e FastText per rappresentare parole in spazi vettoriali continui.
# Pipeline personalizzabili: Gli utenti possono creare pipeline di elaborazione personalizzate e aggiungere i propri componenti.




' python -m spacy download en_core_web_sm '

- idea finale, obbiettivo da perseguire

In [ ]:
# IDEA ALLA BASE: https://medium.com/analytics-vidhya/invoice-information-extraction-using-ocr-and-deep-learning-b79464f54d69
# TUTORIAL CHE STO SEGUENDO PER ADDESTRARE LayoutLM (va addestrato su un DATASET custom):
#  --->  https://youtu.be/bBwDTY38X58?si=sgFvmpoO_x9JlSAi  <----
# Serie di 3 parti scritta come il video: https://medium.com/@shivarama/layoutlmv3-from-zero-to-hero-part-1-85d05818eec4
# Articolo ottimo: https://arxiv.org/abs/2204.08387
# LayoutLMv3 Hugging Face: https://huggingface.co/docs/transformers/model_doc/layoutlmv3
# TESSERACT OCR E Label Studio: https://labelstud.io/blog/improve-ocr-quality-for-receipt-processing-with-tesseract-and-label-studio/

# Altri link utili:
# Classificare documenti con layoutlm: https://www.mlexpert.io/blog/document-classification-with-layoutlmv3
# Teoria su huggingface: https://huggingface.co/blog/document-ai

# Nota --> vale la pena approfondire anche Donut.

Librerie

In [2]:
import pdfplumber
import pdf2image
import pytesseract
from PIL import Image
import cv2
import spacy
import json
import numpy as np
import re
import camelot

# Specificare il percorso di Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Users/TirocinioMatteo/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'  

- Esempio utilizzo Tesseract OCR (non usato nei codici sotto--> da provare)

In [6]:
# Imposta il percorso dell'eseguibile di Tesseract (fatto sopra)
# pytesseract.pytesseract.tesseract_cmd = r'C:/Users/TirocinioMatteo/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'  

# Carica un'immagine
image = Image.open('C:/Users/TirocinioMatteo/Desktop/Matteo/Python_3/Vai/ai/PDF/input_doc/Immagine_cartellone.jpg')

# Utilizza Tesseract per estrarre il testo dall'immagine
text = pytesseract.image_to_string(image, lang='eng+ita')

# Stampa il testo estratto
print(text)

: Beps
A, 10 MINUTI

FELNUOVO DEL GARDA
Va Miano 129




- Esempio modello NLP SpyCy

In [4]:
# Esempio applicazioni modello spaCy:https://spacy.io/models

# Carica il modello NLP 
nlp = spacy.load("en_core_web_sm")

# Analizza il testo
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

# Tokenizzazione
print("Tokenizzazione:")
for token in doc:
    print(token.text)

# Part-of-Speech Tagging e Lemmatizzazione
print("\nPOS Tagging e Lemmatizzazione:")
for token in doc:
    print(f"{token.text} ({token.pos_}): {token.lemma_}")

# Named Entity Recognition (NER)
print("\nNamed Entity Recognition:")
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")

# Dependency Parsing
print("\nDependency Parsing:")
for token in doc:
    print(f"{token.text} -> {token.dep_} -> {token.head.text}")

Tokenizzazione:
Apple
is
looking
at
buying
U.K.
startup
for
$
1
billion

POS Tagging e Lemmatizzazione:
Apple (PROPN): Apple
is (AUX): be
looking (VERB): look
at (ADP): at
buying (VERB): buy
U.K. (PROPN): U.K.
startup (NOUN): startup
for (ADP): for
$ (SYM): $
1 (NUM): 1
billion (NUM): billion

Named Entity Recognition:
Apple (ORG)
U.K. (GPE)
$1 billion (MONEY)

Dependency Parsing:
Apple -> nsubj -> looking
is -> aux -> looking
looking -> ROOT -> looking
at -> prep -> looking
buying -> pcomp -> at
U.K. -> dobj -> buying
startup -> dep -> looking
for -> prep -> startup
$ -> quantmod -> billion
1 -> compound -> billion
billion -> pobj -> for


- Esempio utilizzo Camelot per tabelle

In [5]:
import camelot

# Carica il file PDF
tables = camelot.read_pdf("C:/Users/TirocinioMatteo/Desktop/Matteo/Python_3/Vai/ai/PDF/input_doc/6C554FC0-10D0-4655-B8DA-CDF3C2F32FE6.pdf", pages='all', flavor='stream')
# pages='all' per estrarre tutte le pagine
# flavor='stream' pr tabelle con bordi non definiti chairamente 
# flavor='lattice' per tabelle con bordi definiti

# Stampa il numero di tabelle trovate
print("Trovate", tables.n, "tabelle")

# Visualizza le tabelle estratte
for i, table in enumerate(tables):
    print(f"\nTabella {i + 1}:")
    print(table.df)

# Salva le tabelle estratte in formato CSV
tables.export('tabelle.csv', f='csv', compress=True)


Trovate 4 tabelle

Tabella 1:
                                                   0                 1  2
0                                    Gentile Cliente  MATTEO AMAGLIANI  ,
1  di seguito sono riportati i movimenti effettua...                     
2   informativo e non rappresenta un estratto conto.                     
3  I  movimenti  dove  non  è  indicata  una  dat...                     
4  pagamento è stato bloccato in seguito alla tua...                     
5                              parte dell'esercente.                     
6                                    Lista movimenti                     
7                       dal 01/03/2023 al 31/03/2023                     

Tabella 2:
             0           1          2                     3  \
0         Data        Data  Tipologia                  Nome   
1   operazione   contabile                                    
2   30/03/2023  02/04/2023  Pagamento               Jamaica   
3   29/03/2023  31/03/2023  Pagamento  A

- Lettura testo da PDF: pdfplumber


In [7]:
# Esempio di utilizzo di pdfplumber per estrarre il testo da un PDF
# Apri il file PDF
with pdfplumber.open("C:/Users/TirocinioMatteo/Desktop/Matteo/Python_3/Vai/ai/PDF/input_doc/6E63F9F6-F8D8-44D7-A40B-B7A9BAFDF5B0.pdf") as pdf:
    # Estrai il testo da tutte le pagine
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
        
# Stampa il testo estratto
print(text)

Data: 02/07/2024
Gentile Cliente MATTEO AMAGLIANI,
di seguito sono riportati i movimenti effettuati dal 01/12/2023 al 31/12/2023. Questo documento ha solo scopo
informativo e non rappresenta un estratto conto.
I movimenti dove non è indicata una data contabile sono in attesa di essere contabilizzati. L'importo del
pagamento è stato bloccato in seguito alla tua conferma di concludere l'acquisto e deve essere confermato da
parte dell'esercente.
Lista movimenti
dal 01/12/2023 al 31/12/2023
Data Data Tipologia Nome Descrizione Importo
operazione contabile
31/12/2023 03/01/2024 Pagamento pam PAM LOCAL PADOVA - 29€
31/12/2023 02/01/2024 Pagamento Aspiag Service S.r.l. ASPIAG SERVICE S.R.L.
- 5,97€
PADOVA
31/12/2023 04/01/2024 Pagamento Trenitalia TRENITALIA - PT WL ROMA - 39,1€
Pag. 1 di 5
HYPE S.p.A. segreteria@pec.hype.it C.F e P.IVA 02686590023
P.zza G. Sella, 1 Capitale sociale: € 1.416.885,00 Iscr Registo Imprese C.C.I.A.A
13900 Biella (BI) www.hype.it Monte Rosa Laghi Alto Piemonte REA

Codice n.0: (base)

In [47]:
 # 0 - Estrazione del testo e delle tabelle --> ok anche per multipagina

 # Funzione per estrarre testo e tabelle dal PDF
def extract_text_and_tables_from_pdf(pdf_path):
    text = ""
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Estrazione del testo
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
            # Estrazione delle tabelle
            for table in page.extract_tables():
                tables.append(table)
    return text.strip(), tables

# Funzione per estrarre entità dal testo
def extract_entities(text):
    nlp = spacy.load("en_core_web_sm") 
    doc = nlp(text)
    entities = {}
    for ent in doc.ents: # Qui uso NER (Named Entity Recognition)
        if ent.label_ not in entities:
            entities[ent.label_] = []
        entities[ent.label_].append(ent.text)
    return entities 

# Questo modello spacy è in italiano non mi sembra efficiente...
# Consultare il sito e fare altri tentativi
""" def extract_entities(text):
    nlp = spacy.load("it_core_news_sm")
    doc = nlp(text)
    entities = {}
    for ent in doc.ents:
        label = ent.label_.lower()
        if label not in entities:
            entities[label] = []
        entities[label].append(ent.text)
    return entities """

# Funzione per pulire e strutturare le tabelle estratte
def clean_and_structure_tables(tables):
    structured_tables = []
    for table in tables:
        cleaned_table = []
        for row in table:
            cleaned_row = [re.sub(r'\s+', ' ', cell.strip()) if cell else "" for cell in row]
            cleaned_table.append(cleaned_row)
        structured_tables.append(cleaned_table)
    return structured_tables

def save_data_to_json(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

# Richiamare le varie funzioni
def process_financial_document(pdf_path, output_path):
    text, tables = extract_text_and_tables_from_pdf(pdf_path)
    entities = extract_entities(text)
    structured_tables = clean_and_structure_tables(tables)
    
    # Organizzare i dati finali
    data = {
        "text": text,
        "entities": entities,
        "tables": structured_tables
    }
    save_data_to_json(data, output_path)

' # Funzione per pulire e strutturare le tabelle estratte\ndef clean_and_structure_tables(tables):\n   structured_tables = []\n   for table in tables:\n       cleaned_table = []\n       for row in table:\n           cleaned_row = [re.sub(r\'\\s+\', \' \', cell.strip()) if cell else "" for cell in row]\n           cleaned_table.append(cleaned_row)\n       structured_tables.append(cleaned_table)\n   return structured_tables\n\ndef save_data_to_json(data, output_path):\n   with open(output_path, \'w\', encoding=\'utf-8\') as json_file:\n       json.dump(data, json_file, indent=4, ensure_ascii=False)\n\n# Richiamare le varie funzioni\ndef process_financial_document(pdf_path, output_path):\n   text, tables = extract_text_and_tables_from_pdf(pdf_path)\n   entities = extract_entities(text)\n   structured_tables = clean_and_structure_tables(tables)\n   \n   # Organizzare i dati finali\n   data = {\n       "text": text,\n       "entities": entities,\n       "tables": structured_tables\n   }\n  

Codice n.1: (Camelot)

In [48]:
# 1: Funzione per estrarre testo da PDF --> usa camelot
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

# Funzione per estrarre tabelle dal PDF usando Camelot
def extract_tables_from_pdf(pdf_path):
    tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')  # 'stream' per tabelle senza bordi ben definiti
    structured_tables = []
    for table in tables:
        structured_tables.append(table.df.values.tolist())
    return structured_tables

# Funzione per estrarre entità dal testo
def extract_entities(text):
    nlp = spacy.load("it_core_news_sm")
    doc = nlp(text)
    entities = {
        "organizzazione": [],
        "luogo": [],
        "persone": [],
        "data": [],
        "valuta": []
    }
    for ent in doc.ents:
        if ent.label_ == "ORG":
            entities["organizzazione"].append(ent.text)
        elif ent.label_ == "GPE" or ent.label_ == "LOC":
            entities["luogo"].append(ent.text)
        elif ent.label_ == "PER":
            entities["persone"].append(ent.text)
        elif ent.label_ == "DATE":
            entities["data"].append(ent.text)
        elif ent.label_ == "MONEY":
            entities["valuta"].append(ent.text)
    return entities

# Funzione per pulire e strutturare le tabelle estratte
def clean_and_structure_tables(tables):
    structured_tables = []
    for table in tables:
        cleaned_table = []
        for row in table:
            cleaned_row = [re.sub(r'\s+', ' ', str(cell).strip()) if cell else "" for cell in row]
            cleaned_table.append(cleaned_row)
        structured_tables.append(cleaned_table)
    return structured_tables

# Funzione per organizzare i movimenti finanziari
def organize_movements_table(tables):
    movements = {
        "Titolo tabella": "Lista movimenti dal 01/07/2024 al 31/07/2024",
        "Colonne": ["Data operazione", "Data contabile", "Tipologia", "Nome", "Descrizione", "Importo"],
        "Righe": [],
        "Totali": {
            "entrate": "",
            "uscite": ""
        }
    }
    for table in tables:
        for row in table:
            if "Totale Entrate" in row[0]:
                movements["totals"]["entrate"] = re.findall(r'\d+[.,]?\d*', row[0])[0] + "€"
            elif "Totale Uscite" in row[0]:
                movements["totals"]["uscite"] = re.findall(r'\d+[.,]?\d*', row[0])[0] + "€"
            else:
                movements["rows"].append(row)
    return movements

# Funzione per salvare i dati in formato JSON
def save_data_to_json(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

# Funzione principale per processare il documento finanziario
def process_financial_document(pdf_path, output_path):
    text = extract_text_from_pdf(pdf_path)
    raw_tables = extract_tables_from_pdf(pdf_path)
    entities = extract_entities(text)
    structured_tables = clean_and_structure_tables(raw_tables)
    movements_table = organize_movements_table(structured_tables)
    
    # Organizzare i dati finali
    data = {
        "document_text": text,
        "entities": entities,
        "tables": [movements_table]
    }
    save_data_to_json(data, output_path)

Codice n.2 (Tesseract)

In [62]:

# Funzione per estrarre testo e tabelle da un'immagine
def extract_text_and_tables_from_image(image_path):
    text = ""
    tables = []
    img = Image.open(image_path)
    # Applica l'OCR con Tesseract sull'immagine
    ocr_text = pytesseract.image_to_string(img)
    text += ocr_text + "\n"
    return text.strip(), tables

# Funzione per estrarre testo e tabelle da un PDF
def extract_text_and_tables_from_pdf(pdf_path):
    text = ""
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Estrazione del testo
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
            # Estrazione delle tabelle
            for table in page.extract_tables():
                tables.append(table)
            # Estrazione e OCR delle immagini
            for image in page.images:
                # Ottieni le coordinate dell'immagine
                bbox = (image['x0'], image['top'], image['x1'], image['bottom'])
                # Ritaglia l'immagine dalla pagina
                img = page.within_bbox(bbox).to_image()
                # Applica l'OCR con Tesseract
                ocr_text = pytesseract.image_to_string(img)
                text += ocr_text + "\n"
    return text.strip(), tables

# Funzione per estrarre entità dal testo
def extract_entities(text, lang='en'):
    if lang == 'en':
        nlp = spacy.load("en_core_web_sm")
    elif lang == 'it':
        nlp = spacy.load("it_core_news_sm")
    else:
        raise ValueError(f"Lingua non supportata: {lang}")
    
    doc = nlp(text)
    entities = {}
    for ent in doc.ents: # Qui uso NER (Named Entity Recognition)
        if ent.label_ not in entities:
            entities[ent.label_] = []
        entities[ent.label_].append(ent.text)
    return entities

# Funzione per pulire e strutturare le tabelle estratte
def clean_and_structure_tables(tables):
    structured_tables = []
    for table in tables:
        cleaned_table = []
        for row in table:
            cleaned_row = [re.sub(r'\s+', ' ', cell.strip()) if cell else "" for cell in row]
            cleaned_table.append(cleaned_row)
    return structured_tables

# Funzione per salvare i dati in un file JSON
def save_data_to_json(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

# Funzione principale per processare l'input (PDF o immagine) e generare il file JSON di output
def process_document(input_path, output_path, lang='en'):
    if input_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        # Input è un'immagine
        text, tables = extract_text_and_tables_from_image(input_path)
    elif input_path.lower().endswith('.pdf'):
        # Input è un PDF
        text, tables = extract_text_and_tables_from_pdf(input_path)
    else:
        raise ValueError("Il file di input deve essere un'immagine (.png, .jpg, .jpeg, .bmp) o un PDF (.pdf)")
    
    entities = extract_entities(text, lang)
    structured_tables = clean_and_structure_tables(tables)
    
    data = {
        "text": text,
        "entities": entities,
        "tables": structured_tables
    }
    
    save_data_to_json(data, output_path)

In [66]:
# Esempio di utilizzo
multi_page_pdf = 'C:/Users/TirocinioMatteo/Desktop/Matteo/Python_3/Vai/ai/PDF/0_documentazione/input_doc/hype_account_balance_pdfs/6C554FC0-10D0-4655-B8DA-CDF3C2F32FE6.pdf'
pdf_path = 'C:/Users/TirocinioMatteo/Desktop/Matteo/Python_3/Vai/ai/PDF/0_documentazione/input_doc/hype_account_balance_pdfs/1C579227-CFA7-4297-B47D-1E7F8341514F.pdf'
image_path = 'C:/Users/TirocinioMatteo/Desktop/Matteo/Python_3/Vai/ai/PDF/0_documentazione/input_doc/hype_account_balance_images/all_images/1C579227-CFA7-4297-B47D-1E7F8341514F_page-0001.jpg'

output_path = 'C:/Users/TirocinioMatteo/Desktop/Matteo/Python_3/Vai/ai/PDF/0_documentazione/out_doc/data_pdf_multipagina_2_tesseract.json'
#process_financial_document(image_path, output_path, lang='it')
process_financial_document(multi_page_pdf, output_path)

PDFSyntaxError: No /Root object! - Is this really a PDF?